# Horizon in python

A pure python version of `horizon` with a test dataset. Issue #1.

In [ ]:
import numpy as np
import rasterio as rio
import pandas as pd
import matplotlib.pyplot as plt

from topocalc.horizon import horizon
from topocalc.core_c.topo_core import c_hor1d, pyhorizon

%load_ext line_profiler

In [ ]:
# create a known surface to test with
dx = 30
surf = [100, 80, 75, 85, 70, 50, 64, 65, 85, 90]
distance = dx * np.arange(len(surf))
df = pd.DataFrame([distance, surf], index=['distance', 'surface']).transpose()

# The gold horizon is the points actual horizon index
df['gold_index'] = [0, 3, 3, 9, 9, 6, 8, 8, 9, 9]
hgt = df.surface.values[df.gold_index.values] - df.surface.values
d = df.distance.values[df.gold_index.values] - df.distance.values
print(hgt, d)
df['gold_horizon'] = hgt / np.sqrt(hgt**2 + d**2)
df.loc[np.isnan(df.gold_horizon), 'gold_horizon'] = 0

# IPW horizon function, both calls will produce the same horizon values,
# the c_hor1d has no extra checks before the C code
# Uses a modified hor1f that is looking at the adjacent value
# df['horizon'] = horizon(90, np.array(surf, dtype=np.float).reshape(1, -1), dx)[0,:]
z = np.ascontiguousarray(df.surface.values, dtype=np.float64)
hcos = np.ones_like(z)
c_hor1d(z, dx, True, hcos)
df['horizon'] = hcos

# create a plot
fig, ax = plt.subplots(figsize=(20,8))
df.plot(x='distance', y='surface', marker='o', ax=ax)

for i, row in df.iterrows():
        
    # starting point
    xi = row.distance
    yi = row.surface
    
    xh = xi + d[i]
    yh = yi + hgt[i]
    plt.plot([xi, xh], [yi, yh], 'r--')
        
    # horizon point
    h = dx * row.horizon
    plt.plot([xi, xi+dx], [yi, yi+h], 'b--')
    
    h = dx * row.gold_horizon
    plt.plot([xi, xi+dx], [yi, yi+h], 'g--')
    
df

In [ ]:
# surface
N = len(df.surface)
m = np.repeat(df.surface.values.reshape(1,-1), N, axis=0)

print(m.shape)

# distance to each point
# k=-1 because the distance to the point itself is 0
distance = dx * np.cumsum(np.tri(N, N, k=-1), axis=0)

# height change, tril is needed to ensure the slope returns nan values
height = np.tril(m.T - m)

# slope
slope = height / distance

# horizon location
hor = np.nanargmax(slope[:, :-1], axis=0)
hor = np.append(hor, N-1)

df['horizon_index'] = hor.astype(int)

df['horizon_height_diff'] = df.surface.values[df.horizon_index.values] - df.surface.values
df['horizon_distance_diff'] = dx * (hor - np.arange(0, N))

df['new_horizon'] =  df.horizon_height_diff / np.sqrt(df.horizon_height_diff**2 + df.horizon_distance_diff**2)
df.loc[df['new_horizon'] < 0, 'new_horizon'] = 0
df.loc[np.isnan(df['new_horizon']), 'new_horizon'] = 0

fig, ax = plt.subplots(2, 2, figsize=(10,10))
ax = ax.flatten()
im = ax[0].imshow(m)
ax[0].set_title('surface height')
plt.colorbar(im, ax=ax[0])

im = ax[1].imshow(distance)
ax[1].set_title('distance from point to all other points')
plt.colorbar(im, ax=ax[1])

im = ax[2].imshow(height)
ax[2].set_title('height difference')
plt.colorbar(im, ax=ax[2])

im = ax[3].imshow(slope)
ax[3].set_title('slope to all points')
plt.colorbar(im, ax=ax[3])

# Testing to ensure the above is exactly what is expected
# for i in range(10):
#     # assert that the distance is right in each column
#     np.testing.assert_equal(distance[i:, i], df.distance[i:] - df.distance[i])

#     # assert the height difference is right in each column
#     np.testing.assert_equal(height[i:, i], df.surface[i:] - df.surface[i])
    
#     # assert the slope is correct
#     slp = (df.surface[i:] - df.surface[i]) / (df.distance[i:] - df.distance[i])
#     np.testing.assert_equal(slope[i:, i], slp)

np.testing.assert_equal(df.gold_horizon.values, df.new_horizon.values)
df

In [ ]:
def pyhorizon_1d(surface, dx):
    
    N = len(surface)
    
    m = np.repeat(surface.reshape(1,-1), N, axis=0)

    # distance to each point
    # k=-1 because the distance to the point itself is 0
    distance = dx * np.cumsum(np.tri(N, N, k=-1), axis=0)

    # height change
    height = np.tril(m.T - m)

    # slope
    slope = height / distance

    # horizon location
    hor = np.nanargmax(slope[:, :-1], axis=0)
    hor = np.append(hor, N-1)
    horizon_index = hor.astype(int)

    horizon_height_diff = surface[horizon_index] - surface
    horizon_distance_diff = dx * (hor - np.arange(0, N))

    new_horizon =  horizon_height_diff / np.sqrt(horizon_height_diff**2 + horizon_distance_diff**2)
    
    new_horizon[new_horizon < 0] = 0
    new_horizon[np.isnan(new_horizon)] = 0
    
    return new_horizon
    
df['pyhorizon'] = pyhorizon_1d(np.array(surf), dx)
assert np.sum(df.pyhorizon.values - df.gold_horizon.values) == 0.0

df

In [ ]:
# plot all the horizons

fig, ax = plt.subplots(figsize=(20,8))
df.plot(x='distance', y='surface', marker='o', ax=ax)

for i, row in df.iterrows():
        
    # starting point
    xi = row.distance
    yi = row.surface
    
    xh = xi + d[i]
    yh = yi + hgt[i]
    plt.plot([xi, xh], [yi, yh], 'r--')
        
    # horizon point
    h = dx * row.horizon
    plt.plot([xi, xi+dx], [yi, yi+h], 'b--')
    
    # gold horizon
    h = dx * row.gold_horizon
    plt.plot([xi, xi+dx], [yi, yi+h], 'g--')
    
    # python calc
    h = dx * row.pyhorizon
    plt.plot([xi, xi+dx], [yi, yi+h], 'k--')
    
df

In [ ]:
# Try this in 2D
# Vectorization does not work well for larger 2D domains due to memory consumption

dxa = 30

# raster
ras = rio.open('data/test_proj_windowed_int.tif')
ar = ras.read(1)
ar = ar.astype(np.float)

# sample dataset
ar = np.repeat(df.surface.values.reshape(1,-1), repeats=15, axis=0)

print(f"original dem: {ar.shape}")

def pyhorizon_vec(dem, dx):
    
    # needs to be a float
    if dem.dtype != np.float64:
        dem = dem.astype(np.float64)
        
    nrows, ncols = dem.shape
    hcos = np.zeros_like(dem)
    horizon_index = np.zeros_like(dem)
    
    # distance to each point
    # k=-1 because the distance to the point itself is 0
    distance = dx * np.cumsum(np.tri(ncols, ncols, k=-1), axis=0)
    distance = np.repeat(distance[..., np.newaxis], nrows, axis=2)
    print(f"distance shape: {distance.shape}")
    col_index = np.arange(0, ncols)
    
    # reshape the DEM into a cube where the 3rd dimensions are the rows
    d = np.rot90(dem).reshape(1, ncols, nrows)
    m = np.repeat(d, ncols, axis=0)
    m = np.fliplr(m)
    print(m.shape)
    
    # np.tril does not work with 3D arrays, use triu indicies to set
    # the heights to 0
    height = m.transpose(1, 0, 2) - m
    tril_idx = np.triu_indices(ncols, k=1, m=ncols)
    height[tril_idx] = 0
    print(f"height shape: {height.shape}")
    
    slope = height / distance
    
    # horizon location
    horizon_index = np.nanargmax(slope[:, :-1, :], axis=0)
    horizon_index = np.vstack([horizon_index, (ncols - 1) * np.ones((1, nrows))])
    horizon_index = np.rot90(horizon_index)
    print(f"horizon_index shape: {horizon_index.shape}")
#     print(horizon_index)
    hidx = horizon_index.astype(int)
    
#     np.testing.assert_equal(df.horizon_index.values, horizon_index[0,:])
    
#     horizon_height_diff = surface[hidx] - surface
    
#     plt.imshow(horizon_index)
#     plt.colorbar()
    
#     for n in range(ar.shape[0]):
#         surface = dem[n, :]

# #         m = np.repeat(surface.reshape(1,-1), ncols, axis=0)
                
#         # height change
# #         height = np.tril(m.T - m)

#         # slope
# #         slope = height / distance

# #         # horizon location
# #         hor = np.nanargmax(slope[:, :-1], axis=0)
# #         hor = np.append(hor, ncols-1)
# #         hidx = hor.astype(int)

# #         horizon_height_diff = surface[hidx] - surface
#         horizon_distance_diff = dx * (hor - col_index)

#         new_horizon = horizon_height_diff / np.sqrt(horizon_height_diff**2 + horizon_distance_diff**2)

#         new_horizon[new_horizon < 0] = 0
#         new_horizon[np.isnan(new_horizon)] = 0

#         hcos[n, :] = new_horizon
#         horizon_index[n, :] = hidx
        
    return hcos, horizon_index

%lprun -f pyhorizon_vec pyhorizon_vec(ar, dxa)

# process a row at a time
# hcos, horizon_index =  pyhorizon(ar, dxa)


np.testing.assert_equal(df.gold_horizon.values, hcos[0,:])

In [ ]:
# Try this in 2D

# raster
ras = rio.open('data/test_proj_windowed_int.tif')
ar = ras.read(1)
ar = ar.astype(np.float)

# sample dataset
# ar = np.repeat(df.surface.values.reshape(1,-1), repeats=15, axis=0)

dxa = 30

def pyhorizon_2d(dem, dx):
    
    # needs to be a float
    if dem.dtype != np.float64:
        dem = dem.astype(np.float64)
        
    nrows, ncols = dem.shape
    hcos = np.zeros_like(dem)
    horizon_index = np.zeros_like(dem)
    
    # distance to each point
    # k=-1 because the distance to the point itself is 0
    distance = dx * np.cumsum(np.tri(ncols, ncols, k=-1), axis=0)
    col_index = np.arange(0, ncols)
    
    for n in range(nrows):
        surface = dem[n, :]

        m = np.repeat(surface.reshape(1,-1), ncols, axis=0)
        
        # height change
        height = np.tril(m.T - m)

        # slope
        slope = height / distance

        # horizon location
        hor = np.nanargmax(slope[:, :-1], axis=0)
        hor = np.append(hor, ncols-1)
        hidx = hor.astype(int)

        horizon_height_diff = surface[hidx] - surface
        horizon_distance_diff = dx * (hor - col_index)

        new_horizon = horizon_height_diff / np.sqrt(horizon_height_diff**2 + horizon_distance_diff**2)

        new_horizon[new_horizon < 0] = 0
        new_horizon[np.isnan(new_horizon)] = 0

        hcos[n, :] = new_horizon
        horizon_index[n, :] = hidx
        
    return hcos, horizon_index

# %lprun -f pyhorizon_2d pyhorizon_2d(ar, dxa)

# %timeit -n1000 -r3 hcos, horizon_index = pyhorizon_2d(ar, dx)
%timeit -n1 -r1 hcos, horizon_index = pyhorizon_2d(ar, dx)

# process a row at a time
# hcos, horizon_index =  pyhorizon_2d(ar, dxa)

# pyhorizon cython

Compile pyhorizon in cython to see if any speed ups can be gained

In [ ]:
import numpy as np
import rasterio as rio
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

from topocalc.core_c.topo_core import pyhorizon

In [ ]:
# sample dataset
dx = 30.0
surf = [100, 80, 75, 85, 70, 50, 64, 65, 85, 90]
distance = dx * np.arange(len(surf))
df = pd.DataFrame([distance, surf], index=['distance', 'surface']).transpose()

df['gold_index'] = [0, 3, 3, 9, 9, 6, 8, 8, 9, 9]
hgt = df.surface.values[df.gold_index.values] - df.surface.values
d = df.distance.values[df.gold_index.values] - df.distance.values
df['gold_horizon'] = hgt / np.sqrt(hgt**2 + d**2)
df.loc[np.isnan(df.gold_horizon), 'gold_horizon'] = 0

ar = np.repeat(df.surface.values.reshape(1,-1), repeats=15, axis=0)
ar = ar.astype(np.double)

%timeit -n1000 -r3 hcos, horizon_index = pyhorizon(ar, dx)

hcos, horizon_index = pyhorizon(ar, dx)
np.testing.assert_equal(df.gold_horizon.values, hcos[0,:])


In [ ]:
# raster
ras = rio.open('data/test_proj_windowed_int.tif')
ar = ras.read(1)
ar = ar.astype(np.float)
dxa = 30.0

%timeit -n1 -r1 hcos, horizon_index = pyhorizon(ar, dxa)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,10))
im = ax[0].imshow(ar)
plt.colorbar(im, ax=ax[0])
im = ax[1].imshow(hcos)
plt.colorbar(im, ax=ax[1])
im = ax[2].imshow(horizon_index)
plt.colorbar(im, ax=ax[2])